# Introduction

This tutorial will introduce the basic functionality and usage of MDAnalysis. It will focus on analyzing a Lithium-ion battery electrolyte composed of 49 Li<sup>+</sup>, 49 PF<sub>6</sub><sup>-</sup>, 200 Ethylene Carbonate, and 200 Butyro Nitride.

If you are unfamiliar with the MDAnalysis Universe and AtomGroup, you should check out the tutorial [here]{https://www.mdanalysis.org/MDAnalysisTutorial/basics.html#}. This tutorial will assume a basic familiarity with those data structures.

## Creating a Solution


First, we import solvation_analysis and MDA and instantiate a universe object. 

In [14]:
# imports
import solvation_analysis
import MDAnalysis as mda

# define paths to data
data = "../solvation_analysis/tests/data/bn_fec.data"
traj = "../solvation_analysis/tests/data/bn_fec_short_unwrap.dcd"

# instantiate Universe
u = mda.Universe(data, traj)

Second, we need to select the AtomGroups for the solute and solvents that will compose the solution. In this tutorial, I am selecting the relevant AtomGroups by the Atom types but you can select the AtomGroups in any way you like. Generally, solvation_analysis will work best when 1) the solute and solvents are disjoint sets and 2) the set of solvents includes all atoms that are not the solute.

In [15]:
# define solute AtomGroup
li_atoms = u.atoms.select_atoms("type 22")

# define solvent AtomGroups
pf6 = u.atoms.select_atoms("byres type 21")
bn = u.atoms.select_atoms("byres type 5")
fec = u.atoms.select_atoms("byres type 19")

Finally, we can instantiate the Solution from the solute and solvents! The solvents are supplied with a dict of {str: AtomGroups}. The strings should be the names of the solvents or some other convenient identifier because Solution will use this to identify the solvent in all future analysis.

In [16]:
# instantiate solution
solution = Solution(li_atoms, {'pf6': pf6, 'bn': bn, 'fec': fec})

We can see that solution now contains our atoms of interest as attributes:

In [18]:
print(solution.solute)
print(solution.solvents['bn'])

<AtomGroup [<Atom 6734: of type 22 resid 651 and segid SYSTEM>, <Atom 6742: of type 22 resid 652 and segid SYSTEM>, <Atom 6750: of type 22 resid 653 and segid SYSTEM>, ..., <Atom 7102: of type 22 resid 697 and segid SYSTEM>, <Atom 7110: of type 22 resid 698 and segid SYSTEM>, <Atom 7118: of type 22 resid 699 and segid SYSTEM>]>
<AtomGroup [<Atom 1: of type 1 resid 1 and segid SYSTEM>, <Atom 2: of type 2 resid 1 and segid SYSTEM>, <Atom 3: of type 3 resid 1 and segid SYSTEM>, ..., <Atom 4354: of type 10 resid 363 and segid SYSTEM>, <Atom 4355: of type 11 resid 363 and segid SYSTEM>, <Atom 4356: of type 12 resid 363 and segid SYSTEM>]>


## Running and Validating a Solution

Now that we have a solution, we can run the analysis! This follows the typical MDA analysis interface of calling `analysis.run()`. When `run` is called, a few things happen:


First, Solution calculates the RDF between the solute and each solvent and
uses it to identify the radius of the first solvation shell.

Second, Solution finds all atoms in the first solvation shell, using
the cutoff radii for each solvent. This analysis is repeated for each solute at every frame in the
analysis and the data is compiled into a pandas.DataFrame.

Finally, Solution instantiates Speciation, Coordination, and Pairing
objects from the solvation_data, providing a convenient interface to
further analysis. These will be discussed later in the tutorial.

Now let's do it!

In [19]:
solution.run()

/Users/orioncohen/miniconda3/envs/solvation_analysis/lib/python3.8/site-packages/solvation_analysis-0.0.0+121.gb238845.dirty-py3.8.egg/solvation_analysis/rdf_parser.py:159: UserWarning: No solvation shell detected.
  warnings.warn("No solvation shell detected.")
/Users/orioncohen/miniconda3/envs/solvation_analysis/lib/python3.8/site-packages/solvation_analysis-0.0.0+121.gb238845.dirty-py3.8.egg/solvation_analysis/solution.py:186: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pairs[:, 1] = solvent.ids[[pairs[:, 1]]]


In [20]:
solution.radii

{'pf6': nan, 'bn': 2.614678772286192, 'fec': 2.431583419310968}

When `run` is called, a few things happen:


First, Solution calculates the RDF between the solute and each solvent and
uses it to identify the radius of the first solvation shell.

Second, Solution finds all atoms in the first solvation shell, using
the cutoff radii for each solvent. This analysis is repeated for each solute at every frame in the
analysis and the data is compiled into a pandas.DataFrame.

Finally, Solution instantiates Speciation, Coordination, and Pairing
objects from the solvation_data, providing a convenient interface to
further analysis. These will be discussed later in the tutorial.

## Basic Analysis with Coordination and Pairing

## Exploratory Analysis with Speciation